## Part 3: ElasticSearch

In part 2 we have performed streaming sentiment analysis. The Belgian dealer is pleased to see the results. Nevertheless he wants to go a step further. He wants to create a search engines:
1) A search engine on his website to search for cars
2) An internal search engine in which he can search the tweets for German cars

The car dealer wants to be convinced of the power of ElasticSearch and askes you to run 15 queries. Are you able to construct these queries?

### 3a) Indexes

There are already two indexes (with types) available in ElasticSearch:
* **/cars/cars_db**: An index/type containing the cars for sale on the Belgian market. This index is the result of the individual assignment.
* **cars_twitter/tweets**: An index/type containing tweets about German cars. This index is the result of part 2 in this assignment.

You first job is to look at the mappings. Can you answer the following questions:
* What type is the field **age** in the /cars/cars_db index?
 AGE field is "text"
* What type is the field **brandName** in the /cars/cars_db index?
 BRANDNAME field is "text"
* What does the following key-value pair mean?
 A key-value pair (KVP) is an abstract data type that includes a group of key identifiers and a set of associated values. Key-value pairs are frequently used in lookup tables, hash tables and configuration files.
* What does **"ignore_above: 256"** mean?
Strings longer than the ignore_above setting will not be indexed or stored

## 3b) Searching

In this part, you need to perform queries in ElasticSearch (Kibana). Each query needs to be pasted into the %elasticsearch cell. In the %md cell "Hits" the number of hits should be entered. Finally, if an additional qestion is asked, you should answer in the %md cell with "Answer".

### Question 1 (Example):
Create a query that returns all the documents in the cars_twitter/tweets index.
Was this easy?

In [1]:
%elasticsearch
GET /cars_twitter/tweets/_search
{
  "query": {
    "match_all": {}
  }
}

UsageError: Line magic function `%elasticsearch` not found.


#### Hits
6538

### Question 2:
Create a query that return all the **oldtimers** in the **cars/cars_db** index.

In [1]:
%elasticsearch
####here we filtered the data based on age_group and value old timer
GET /cars/cars_db/_search
{
  "query": {
    "match": {
      "age_group": "oldtimer"
      }
    }
  }

UsageError: Line magic function `%elasticsearch` not found.


#### Hits

14491

### Question 3:
Find all the German cars in the cars/cars_bd index.

In [ ]:
%elasticsearch
####we have the list of german cars 
GET /cars/car_db/_search
{
  "query": {
    "terms": {
      "brandName.keyword": [ "Audi", "BMW", "Mercedes", "Opel", "Porsche", "Volkswagen" ]
    }
  }
}

#### Hits
13264

### Question 4:

Return all the cars with a **price** between 20000 and 30000.

In [ ]:
%elasticsearch
GET /cars/cars_db/_search
{
  "query": {
    "range": {
      "sellingPrice": {
        "gte": 20000,
        "lte": 30000
      }
    }
  }
}

#### Hits
5327

### Question 5:

Return the models of which the name only consist of numbers. (Hint: **regexp** query)

In [ ]:
%elasticsearch

####we did regex filter for the numeric and also did not include characters
GET /cars/cars_db/_search
{
  "query": {
    "bool":{
      "filter": [
        {
        "regexp": {
          "modelName": "[0-9]*"
          }
        }
      ],
      "must_not": [
        {
        "regexp": {
          "modelName": "[a-zA-Z]*"
          }
        }
      ]
    }
  }
}

#### Hits

2932

### Question 6:
Return the tweets that have a **brand** field that is not null.

In [ ]:
%elasticsearch

####here we use exist function
GET /cars_twitter/tweets/_search
{
  "query": {
    "exists": {
      "field": "brand"
    }
  }
}

#### Hits
500

### Question 7:

Return the tweets that contain **Mercedes** (case insensitive) in their text body.

How come the car with id "AV-XehVbcAXvn561YE47" is include in the results, while no brand field is present for this car?

In [3]:
%elasticsearch
GET /cars_twitter/tweets/_search
{
  "query": {
    "match": {
      "text": "mercedes"
    }
  }
}

UsageError: Line magic function `%elasticsearch` not found.


#### Hits
81


#### Answer
Because we are only searching for tweets that have the word "Mercedes" in the text body without specifying anything about the brand name.

### Question 8:

Return the tweets that have **audi** (case insensitive) in their **hashtags** or **mentions**

In [ ]:
%elasticsearch
####since we 
{ 
  "query": { 
    "bool": { 
      "should": [{ "match": { "entities_hashtags.text":"audi" } }, 
                   { "match": { "entities_user_mentions.text":"audi" } }
                 ] 
    } 
  } 
} 

#### Answer
5

### Question 9:

Create a query that returns the tweets with the exact phrase **"sports car"** (case insensitive) in their text.

In [ ]:
%elasticsearch

####there is no results for this query###########
GET /cars_twitter/tweets/_search 
{ 
    "query": { 
      "bool":{
        "should": [{"match_phrase" : { "text" : "sports car" } },
                   {"match_phrase" : { "text" : "SPORTS CAR" } }]
      }
    }
}

#### Hits
0

### Question 10:
Return the tweets with **BMW** in their body that have a **positive sentiment**.

In [ ]:
%elasticsearch

#####we ran a search to include BMW and also postive sentiment
GET /cars_twitter/tweets/_search 
{ 
  "query": { 
    "bool": { 
      "must": [ 
        { 
          "match": { "text":"BMW" } 
        }, 
        { 
          "match": { "sentiment":"positive" } 
        }        
      ] 
    } 
  } 
} 

#### Hits
16

### Question 11:
In ElasticSearch you can accomplish the same result with multiple queries. Can you create a query that returns the same results as question 10, while another query is executed?

What is the difference in reasoning between the query you executed in question 10 and 11?

In [ ]:
%elasticsearch

GET /cars_twitter/tweets/_search 

{ 
  "query": { 
    "bool": { 
      "must": [ 
        { 
          "match": { "text":"BMW" } 
        }, 
        { 
          "term": { "sentiment":{"value":"positive" }} 
        }        
      ] 
    } 
  } 
} 

#### Hits

16 

### Question 12
Return tweets that contain **Audi** that do **not** have a **positive sentiment**. Give priority to tweets that contain **R8** without excluding other tweets.

In [ ]:
%elasticsearch

############ first we filtered on Audi and sentiment not positive and we did a boosted to 2 for R8 
GET /cars_twitter/tweets/_search  

{ 
  "query": { 
    "bool": { 
      "must": [ 
        { 
          "match": { "text":"Audi" } 
        } 
      ], 
      "must_not": [ 
        { 
          "match": { "sentiment":"positive" } 
        } 
      ], 
      "should": [ 
        { 
          "match": { 
            "text": { 
               "query": "R8", 
                "boost": 2 
            } 
          } 
        } 
      ] 
    } 
  } 
} 

 

#### Hits
36

### Question 13
Similarly to question 12, find tweets about **Audi**, tweeted from **Singapore**, and have **no negative sentiment**. Give priority to tweets that contain **R8** without excluding other tweets.

In [ ]:
%elasticsearch

#####we ran the query but there were 0 results ##############
GET /cars_twitter/tweets/_search 
{ 
  "query": { 
    "bool": { 
      "must": [ 
        { 
          "match": { "text":"Audi" } 
        }, 
        { 
          "match": { "user_location":"Singapore" } 
        } 
      ], 
      "must_not": [ 
        { 
          "match": { "sentiment":"negative" } 
        } 
      ], 
      "should": [ 
        { 
          "match": { 
            "text": { 
               "query": "R8", 
                "boost": 2 
            } 
          } 
        } 
      ] 
    } 
  } 
} 

### Question 14
Create a query that returns the **young German cars** that do **not** have a **mileage higher or equal to 10000 km**. Give priority to powerful cars with an **engine power greater or equal to 150 kW** without excluding other results.
Note that we want to use this result on the webpage. We only want to show **3 variabels: "brandName", "modelName", and "sellingPrice"**. Additionally we want to use pagination and show only the second page showing results **10 - 20**.

In [ ]:
%elasticsearch
GET /cars/cars_db/_search
{
  "_source": ["brandName","modelName","sellingPrice"],
  "from": 10,
  "size": 11,
  "query": {
    "bool": {
      "must": [
    {
    "match": {
      "age_group": "young"
      }
    },
    {
        "terms": {"brandName": ["porsche", "opel", "volkswagen", "audi", "mercedes", "bmw"]
    }}],      
    {
    "range": {
      "mileageValue": {
        "lt":10000
        }
      }
    },
    {
    "range": {
      "enginePowerValue": {
        "gte":150,
         "boost": 2
            }
          }
        }
      ]
    }
  }
}

#### Hits
3

### Question 15
Finally you should run a query that returns the tweets having **english** as language and contain the phrase **Porche Boxster**. Note that the words "Porsche" and "Boxster" should be close to each other and **maximum 1 word** can be in between the two words. Additionally the tweets may **NOT** have a **negative sentiment**. For esthetic reasons, the searched terms (Porsche Boxster) should be **highlighted** in bold.

In [ ]:
%elasticsearch
GET /cars_twitter/tweets/_search
{
  "query": {
    "bool": { 
      "must": [
     {"match": {"lang": "en"}},
    {
    "match_phrase": {
      "text": {
        "query": "Porsche Boxster",
        "slop": 1
       }
      }
    }
  ],
      "must_not": [
        {
          "match": { "sentiment":"negative" } 
        }
      ]
    }
  }, 
  "highlight": {
    "fields": {
      "text": {}
    }
  }
}

#### Hits
0

<img src="https://i.ytimg.com/vi/P4LhWSN3YSw/maxresdefault.jpg" />